In [1]:
# 국토교통부 버스노선(서울, 부산 제외)
import requests, bs4
import mysql.connector
from lxml import html
from urllib.request import Request,urlopen
from urllib.parse import urlencode,quote_plus
from bs4 import BeautifulSoup
from xml.etree.ElementTree import parse
from ServiceKey import public_serviceKey

mydb = mysql.connector.connect(
        host = "localhost",
        user = "root", 
        password = "1234",
        database = "travel_supplies_in_pockets"
    )

cursor = mydb.cursor(prepared=True)
citycode = '31150'

# DB에서 도시 코드 구해오기
# 2021.09.17 insert 문 에러로 데이터 삽입 실패
query = ("select citycode from public_busctycodelist where citycode >= {}".format(citycode))

cursor.execute(query)

location_list = []

for i in cursor:
    location_list.append(i)
    
b_cnt = 0
for j in location_list:
    location = j
    if b_cnt == 1:
        mydb.close()
        break
    
    for i in range(1,121):
        routeNo = i
        print("location : {} | {}번째 조회시작".format(location[0],routeNo))    
        url = 'http://openapi.tago.go.kr/openapi/service/BusRouteInfoInqireService/getRouteNoList'
        queryParams = '?' + urlencode({
            quote_plus('serviceKey') : public_serviceKey(),
            quote_plus('cityCode') : location[0],
            quote_plus('routeNo') : routeNo,
            quote_plus('numOfRows') : '10000'
        })

        response = requests.get(url + queryParams).text.encode('utf-8')
        xmlobj = bs4.BeautifulSoup(response, 'lxml-xml')

        rows = xmlobj.findAll('item')
        resultCode = xmlobj.findAll('resultCode')
        if resultCode[0].text == '99':
            print("location : {} | {} 번째부터 조회수 제한이 걸렸습니다.".format(location[0],routeNo))
            b_cnt += 1
            break
        if rows == []:
            print("{} 번쨰데이터가없습니다.".format(routeNo))
            continue
        for j in rows:
            item = j
            routeno = item.findAll('routeno')
            if routeno == []:
                routeno = None
            else:
                routeno = routeno[0].text

            routeid = item.findAll('routeid')
            if routeid == []:
                routeid = None
            else:
                routeid = routeid[0].text

            routetp = item.findAll('routetp')
            if routetp == []:
                routetp = None
            else:
                routetp = routetp[0].text

            startnodenm = item.findAll('startnodenm')
            if startnodenm == []:
                startnodenm = None
            else:
                startnodenm = startnodenm[0].text

            startvehicletime = item.findAll('startvehicletime')
            if startvehicletime == []:
                startvehicletime = None
            else:
                startvehicletime = startvehicletime[0].text

            endnodenm = item.findAll('endnodenm')
            if endnodenm == []:
                endnodenm = None
            else:
                endnodenm = endnodenm[0].text

            endvehicletime = item.findAll('endvehicletime')
            if endvehicletime == []:
                endvehicletime = None
            else:
                endvehicletime = endvehicletime[0].text
                
            sql = "INSERT INTO public_busroute_info (routeno, routeid, routetp, startnodenm, startvehicletime, endnodenm, endvehicletime,citycode) VALUES (?,?,?,?,?,?,?,?)"
            val = (routeno, routeid, routetp, startnodenm, startvehicletime, endnodenm, endvehicletime, location[0])


            cursor.execute(sql, val)
            mydb.commit()

location : 31150 | 1번째 조회시작
location : 31150 | 1 번째부터 조회수 제한이 걸렸습니다.
